In [ ]:
####################
### Dependencies ###
####################

import requests as rq
from dotenv import load_dotenv
import os
import json
import pandas as pd
from io import StringIO
from IPython.display import display
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import sqlite3 as sql
import psycopg as pg
from dotenv import load_dotenv
import asyncio
import nest_asyncio

In [ ]:
#####################
### General Setup ###
#####################

nest_asyncio.apply()
load_dotenv()

In [ ]:
###########################
### Setup PostgreSQL DB ###
###########################

# Make a connection to the database
activateCon = lambda : pg.Connection.connect(os.getenv("DB_URI"), cursor_factory=pg.ClientCursor)
con = activateCon()
cur = con.cursor()

# Initialize transformed tables AFTER IMPORTING THE GEOJSON FILE
cur.execute('''
    CREATE EXTENSION postgis;

    CREATE TABLE jerryTransform AS
    SELECT
        ogc_fid AS id,
        ST_CollectionExtract(ST_MakeValid(ST_GeomFromWKB(wkb_geometry))) AS geo,
        substring(geoid from 1 for 2)::integer AS state,
        substring(geoid from 3 for 3)::integer AS county,
        votes_dem AS dem,
        votes_rep AS rep,
        votes_total AS tot
    FROM jerry;
''')
con.commit()
con.close()

In [ ]:
con = activateCon()
cur = con.cursor()

cur.execute('''
    create table trans2 as
    select *, ST_ClusterKMeans(
            ST_Force4D(
                ST_Force3DZ(ST_GeneratePoints(geo, 1, (100000*random()+1)::int), 0.15*random()),
                mvalue => 100000*random() -- set clustering to be weighed by population
            ),
            880
        ) over () as cluster1
    from jerryTransform
    where state = 39;
''')